In [42]:
import pandas as pd

In [64]:
routes_to_neighborhoods = pd.read_csv('data/routes_to_neighborhoods_new.csv')
travel_times = pd.read_csv('data/travel_times.csv')

In [65]:
travel_times['route_id'] = travel_times['route_id'].apply(lambda x: '746' if str(x).startswith('746_') else x)
route_ids = routes_to_neighborhoods['route_id']
filtered_travel_times = travel_times[travel_times['route_id'].isin(route_ids)]

In [66]:
print(filtered_travel_times[['route_id', 'scheduled_travel_time', 'travel_time']])

       route_id scheduled_travel_time travel_time
0            26              00:10:00    00:10:06
1            21              00:14:00    00:13:53
2            42              00:11:00    00:10:30
3            24              00:21:00    00:19:33
4            24              00:24:00    00:22:29
...         ...                   ...         ...
333298       86              00:53:00    00:51:39
333299       86              01:00:00    00:51:53
333300       86              00:34:00    00:33:24
333301       86              00:48:00    00:45:01
333302       86              00:48:00    00:49:08

[242839 rows x 3 columns]


In [67]:
filtered_travel_times['scheduled_travel_time'] = pd.to_datetime(filtered_travel_times['scheduled_travel_time'], format='mixed')
filtered_travel_times['travel_time'] = pd.to_datetime(filtered_travel_times['travel_time'], format='mixed')
filtered_travel_times['lateness'] = (filtered_travel_times['travel_time'] - filtered_travel_times['scheduled_travel_time']).dt.total_seconds() / 60
average_lateness_by_route = filtered_travel_times.groupby('route_id')['lateness'].mean()
sorted_routes = average_lateness_by_route.sort_values(ascending=True)
five_least_late = sorted_routes.head(5)
five_most_late = sorted_routes.tail(5).sort_values(ascending=False)
most_late_route_ids = five_most_late.index.get_level_values('route_id').tolist()
least_late_route_ids = five_least_late.index.get_level_values('route_id').tolist()

print(most_late_route_ids)
print(least_late_route_ids)

C:\Users\munir\AppData\Local\Temp\ipykernel_4672\3850961901.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_travel_times['scheduled_travel_time'] = pd.to_datetime(filtered_travel_times['scheduled_travel_time'], format='mixed')


['746', 'CT2', '92', '121', '426']
['428', '14', '86', '505', '171']


C:\Users\munir\AppData\Local\Temp\ipykernel_4672\3850961901.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_travel_times['travel_time'] = pd.to_datetime(filtered_travel_times['travel_time'], format='mixed')
C:\Users\munir\AppData\Local\Temp\ipykernel_4672\3850961901.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_travel_times['lateness'] = (filtered_travel_times['travel_time'] - filtered_travel_times['scheduled_travel_time']).dt.total_seconds() / 60


In [74]:
import ast

neighborhoods_most_late = routes_to_neighborhoods[routes_to_neighborhoods['route_id'].astype(str).isin(most_late_route_ids)]['neighborhood']
neighborhoods_least_late = routes_to_neighborhoods[routes_to_neighborhoods['route_id'].astype(str).isin(least_late_route_ids)]['neighborhood']

neighborhoods_most_late_list = [ast.literal_eval(neigh_str) for neigh_str in neighborhoods_most_late]
neighborhoods_most_late_list = set([neighborhood for sublist in neighborhoods_most_late_list for neighborhood in sublist])
neighborhoods_least_late_list = [ast.literal_eval(neigh_str) for neigh_str in neighborhoods_least_late]
neighborhoods_least_late_list = set([neighborhood for sublist in neighborhoods_least_late_list for neighborhood in sublist])

print(neighborhoods_most_late_list)
print(neighborhoods_least_late_list)
print(neighborhoods_least_late_list.intersection(neighborhoods_most_late_list))

{'Downtown', 'South Boston Waterfront', 'Longwood', 'Mission Hill', 'Roxbury', 'East Boston', 'Charlestown', 'Fenway'}
{'Hyde Park', 'Downtown', 'Roslindale', 'Dorchester', 'Mission Hill', 'Roxbury', 'Charlestown', 'Brighton', 'Jamaica Plain', 'East Boston', 'Allston', 'South Boston'}
{'Downtown', 'Mission Hill', 'Roxbury', 'East Boston', 'Charlestown'}
